# Microns morphology effect part 3

This analysis continues the notebook titled "microns morphology effect" and "microns morphology effect - part 2". In those notebooks we demonstrated that in the MICrONS connectome connections are not formed statistically independently. Specifically: If a connection exists from neuron i to neuron j, then the probability that a connection exists from i to the nearest neighbor of j is increased. We also demonstrated that this can only partially be explained by a "global" effect based on long-tailed degree distributions. Instead, the effect has a complex spatial structure.

**We suggest you first look at those two notebook if you have not already.**

### More on the spatial structure of the morphology effect
Here we investigate the spatial structure further.

The previous analyses demonstrated dependence between a connection i -> j and i -> nearest neighbor of j. But what about other neurons around j? How far does the effect reach? Additionally, it is conceivable that innervation of one location makes innervation of another location **less** likely.

We explore this in the following way. Instead of an overall mean, we consider the connection probability of *individual neurons* into given spatial bins. Then we consider the correlations of connection probabilities into pairs of spatial bins over individual neurons. That is, we ask: If a neuron has a high connection probability into a spatial bin at one location, is its connection probability into another bin increased or decreased? 

As in part 2, such an effect can be global, where an increase into all other bins is observed, and which can be explained by degree distributions; or local, with spatial structure and not captured by a configuration model.

### Similar to part 2 - but different
You will note that the structure of this notebook and its analyses is similar to part 2. But there are important differences. So please read the explanations and code carefully.

## Setup
We begin by importing relevant packages and setting up paths.

### Customization options
We have set up this notebook and the paths below for the analysis of the excitatory subgraph of the MICrONS connectome.

But you can use it to analyze other connectomes as well, for example of the various circuit models we offer on our platform. You can download the .h5-formatted connectome from the platform and update the paths below accordingly. Contact us if you want assistance with this.

### Caching the results
This analysis is somewhat computationally expensive (~20 minutes for 50k neuron connectomes). Hence, we broke it up into two parts: First, we calculate a pandas.DataFrame that holds pre-digested data. Then we plot the data according to customizable specifications. 

The output of the first, expensive step is saved into a file, such that it does not have to be re-calculated: In future executions it is instead read from the file, unless you set the "force_recalculation" flag to True. 

Note that we have initialized the default cache file with all relevant results for your convenience. 

In [ ]:
import numpy
import pandas
import h5py
import os

import conntility

from scipy.spatial.distance import cdist
import tqdm

from matplotlib import pyplot as plt
from ipywidgets import widgets, interactive

# Location of the file holding the connectome information
fn_connectivity = "../../../../shared_data/MICrONS_SONATA/microns_mm3_connectome_EXC_from_sonata.h5"

# Location of the file for storing/caching results of the compuations. If it does not exists, it will be created.
# The file below already has the relevant results pre-calculted.
fn_digest_dataframe = "../../../../shared_data/MICrONS_SONATA/microns_EXC_nn_morphology_effect_digest.h5"


# If you set this flag to True, expensive calculations will be repeated even if the result already exists in the cache file above.
# Note: If you set this to True you will have to update "fn_digest_dataframe" to a different, local path. This is because the 
# default location of "fn_digest_dataframe" is on a read-only file system.
force_recalculation = False


## Selecting the connection matrix
We read a connection matrix from an .h5 file. However, a file can contain more than one connection matrix. 

**Select the one to analyze from the dropdown.**

If you have not updated "fn_connectivity" in the previous cell, you will find the following connection matriced:
  - connectivity/data: The latest version of MICrONS
  - connectivity/dist_dep: A distance-dependent control fitted against the MICrONS data
  - connectivity/configuration_model: A configuration model control of the MICrONS data

We suggest that you start with "connectivity/data". Then, as a comparison, also try "connectivity/configuration_model". Finally, you can try "connectivity/dist_dep".

In [ ]:
with h5py.File(fn_connectivity, "r") as h5:
    contents = []
    prefixes = list(h5.keys())
    for prefix in prefixes:
        [contents.append(prefix + "/" + _k) for _k in h5[prefix].keys()]

sel_matrix = widgets.Dropdown(options=contents, index=1, description="Select matrix")
display(sel_matrix)

We load the selected connectivity matrix.

In [ ]:
selected = str(sel_matrix.value)
M = conntility.ConnectivityMatrix.from_h5(fn_connectivity, 
                                          prefix=sel_matrix.value.split("/")[0],
                                          group_name=sel_matrix.value.split("/")[1])
print(f"Loaded {len(M)} nodes with {len(M.edges)} edges from {selected}")

col_y = "y"
col_xz = ["x", "z"]
# This flag indicates whether y indicates a "depth", i.e. is counted from the top of L1 (True), or a reverse depth, counted from the bottom of L6 (False).
# This is relevatn purely for plotting purposes at the end.
y_is_depth = True


### Generate spatial bins
We generate spatial bins for the offset of neurons pairs along the y-axis, and their distance in the x/z-plane.
For the y-bins we ensure that they are centered around 0.

We found 50 um to be a good bin size, but you can update it. In that case, you will have to set the "force_recalculate" flag to True.

In [ ]:
# It is possible to adjust the bin size of the final plots here.
bin_sz = 50.0 # um


def make_spatial_bins(M_h, cols, bin_sz):
    _data = M_h.vertices[cols]
    delta = _data.max() - _data.min()

    sz = numpy.sqrt((delta.values ** 2).sum())
    if len(delta) == 1: # case 1d: negative and positive bins
        bins = numpy.arange(0, (bin_sz * numpy.ceil(sz / bin_sz)) + bin_sz, bin_sz)
        bins = numpy.hstack([-bins[:0:-1], bins])
    else: # case 2d: Only positive bins, but exclude 0 dist
        bins = numpy.arange(0, (bin_sz * numpy.ceil(sz / bin_sz)) + bin_sz, bin_sz)
        bins = numpy.hstack([[0, 1E-12], bins[1:]])
    return bins

dbins_xz = make_spatial_bins(M, col_xz, bin_sz)
binid_xz = numpy.arange(0, len(dbins_xz) + 1)

dbins_y = make_spatial_bins(M, [col_y], bin_sz)
binid_y = numpy.arange(0, len(dbins_y) + 1)

bin_centers_y = 0.5 * (dbins_y[:-1] + dbins_y[1:])
bin_centers_xz = 0.5 * (dbins_xz[1:-1] + dbins_xz[2:])


## Main calculation

This is where this notebook differs from "microns morphology effect part 2". Although we still break up the population into chunks and execute the analysis chunk by chunk.

### Step 1
Here, we calculate a DataFrame with the following structure:

Each row represents a possible combination of:
  - A numeric identifier of a presynaptic neuron (i)
  - spatial bin of an offset from neuron i in the xz plane
  - spatial bin of an offset along the y-axis from neuron i
  - number of edges (synapses) from i to a neuron j in the indicated spatial bin

The first four columns list the values of these four properties. **a fifth column then counts the number of pairs of neurons for that combination**.

The difference to "microns morphology effect part 2" is that we keep the counts for individual presynaptic neurons separate, instead of summing them.

Additionally, we calculate a second DataFrame where i instead indicates a postsynaptic neuron and the number of edges from j to i is counted.

### Step 2
We then use this DataFrame to calculate the following: For each pre- (post-) synaptic neuron the number of potential synaptic partners it has in each spatial bin. The connection probability of each neuron into each spatial bin. This connection probability is calculated separately for different minimum edge counts, i.e. if 1, 2, 3, ... edges is the minumum for considering a pair of neurons connected. This allows focusing on the structure of exceptionally strong connections.

### Caching the results
As before, if the result has already been calculated before, it is read from a file instead. Otherwise it is stored into the file.

In [ ]:
mat = M.matrix.tocsr()

def for_chunk(chunk, direction="outgoing"):
    # Which offset bin the pairs fall into
    if direction == "outgoing": # means: from chunk to all neurons
        Dxz = cdist(M.vertices.iloc[chunk][col_xz], M.vertices[col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M.vertices.iloc[chunk][[col_y]].values - M.vertices[[col_y]].values.transpose() # PRE X POST
        # PRE - POST. If y_is_depth: Negative values -> downwards connection.
        Dy = numpy.digitize(Dy, dbins_y) - 1
        
        j, i = numpy.meshgrid(numpy.arange(len(M)), chunk) # chunk is i: presyn. => consider outgoing
        node_id = i.flatten()
        edge_count = mat[chunk].toarray().flatten()
    elif direction == "incoming":
        Dxz = cdist(M.vertices[col_xz], M.vertices.iloc[chunk][col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M.vertices[[col_y]].values - M.vertices.iloc[chunk][[col_y]].values.transpose() # PRE X POST
        # PRE - POST. If y_is_depth: Negative values -> downwards connection.
        Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
        
        j, i = numpy.meshgrid(chunk, numpy.arange(len(M))) # chunk is j: postsyn. => consider incoming
        node_id = j.flatten()
        edge_count = mat[:, chunk].toarray().flatten()
    
    # Count instances of each
    ret = pandas.DataFrame({
        "xz": Dxz.flatten(),
        "y": Dy.flatten(),
        "edge_count": edge_count,
        "node_id": node_id,
    }).value_counts()
    ret.name = "count"
    return ret

def update_xz_y_bins(df):
    cols = list(df.index.names)
    df = df.reset_index()
    df["xz"] = bin_centers_xz[df["xz"]]
    df["y"] = bin_centers_y[df["y"]]
    df = df.set_index(cols)
    return df["count"]

### Try the cache first

Here, we test whether the result can already be found in the cache ("digest") file. If so and "force_recalculation" is False, we load it.

Otherwise, we iterate over chunks of neurons performing the costly calculation. This may take 20-30 minutes.

In [ ]:
digest_exists = False
if not force_recalculation:
    if os.path.isfile(fn_digest_dataframe):
        with h5py.File(fn_digest_dataframe, "r") as h5:
            if selected + "/P_in" in h5 and \
                selected + "/N_in" in h5 and \
                selected + "/P_out" in h5 and \
                selected + "/P_in" in h5:
                    digest_exists = True

if digest_exists:
    P_in = pandas.read_hdf(fn_digest_dataframe, selected + "/P_in")
    P_out = pandas.read_hdf(fn_digest_dataframe, selected + "/P_out")
    N_in = pandas.read_hdf(fn_digest_dataframe, selected + "/N_in")
    N_out = pandas.read_hdf(fn_digest_dataframe, selected + "/N_out")
else:
    df_for_outgoing = []
    df_for_incoming = []

    chunk_sz = 5000
    chunking = numpy.arange(0, len(M) + chunk_sz, chunk_sz)

    for a, b in tqdm.tqdm(list(zip(chunking[:-1], chunking[1:]))):
        chunk = numpy.arange(a, numpy.minimum(b, len(M)))
        df_for_outgoing.append(for_chunk(chunk, direction="outgoing"))
        df_for_incoming.append(for_chunk(chunk, direction="incoming"))
        
    df_for_outgoing = pandas.concat(df_for_outgoing, axis=0).drop(-1, axis=0, level="xz")
    df_for_incoming = pandas.concat(df_for_incoming, axis=0).drop(-1, axis=0, level="xz")

    df_for_outgoing = update_xz_y_bins(df_for_outgoing)
    df_for_incoming = update_xz_y_bins(df_for_incoming)

    def conn_prob_for_thresholds(df_in):
        tmp = df_in.pivot(index="edge_count", columns="node_id", values="count").fillna(0)
        return (tmp.loc[::-1].cumsum(axis=0) / tmp.sum(axis=0)).stack()

    def simply_count(df_in):
        return df_in.reset_index().groupby("node_id")["count"].sum()

    P_out = df_for_outgoing.reset_index().groupby(["xz", "y"]).apply(conn_prob_for_thresholds)
    P_out = P_out.reorder_levels([2, 0, 1, 3])
    N_out = df_for_outgoing.reset_index().groupby(["xz", "y"]).apply(simply_count)

    P_in = df_for_incoming.reset_index().groupby(["xz", "y"]).apply(conn_prob_for_thresholds)
    P_in = P_in.reorder_levels([2, 0, 1, 3])
    N_in = df_for_incoming.reset_index().groupby(["xz", "y"]).apply(simply_count)

    P_in.to_hdf(fn_digest_dataframe, key=(selected + "/P_in"))
    P_out.to_hdf(fn_digest_dataframe, key=(selected + "/P_out"))
    N_in.to_hdf(fn_digest_dataframe, key=(selected + "/N_in"))
    N_out.to_hdf(fn_digest_dataframe, key=(selected + "/N_out"))

## Optional normalization

The following cell is optional. Set the "perform_normalization" flag to True to execute it.

The normalization subtracts from the connection probabilities of a neuron its mean connection probability over all spatial bins. This is related to the distinction between a "global" and a "local" effect of morphology on connectivity. The global effect is one that can be explained by long-tailed degree distributions of neurons: If connection probability into one spatial bin is high, this indicates that the neuron has a high out-degree. Hence, connection probabilities into all other bins will be globally increased, without strong spatial structure. Any spatial structure to the correlation results on top of that will be attributed to a local effect.

Subtracting from each neuron its mean connection probability will remove the impact of the global effect and allows us to study the two types of effect separately.

In [ ]:
perform_normalization = False

if perform_normalization:
    mn_P_out = P_out.groupby(["edge_count", "node_id"]).mean()
    mn_P_out = mn_P_out[pandas.MultiIndex.from_frame(P_out.index.to_frame()[["edge_count", "node_id"]])]
    mn_P_out.index = P_out.index
    mn_P_in = P_in.groupby(["edge_count", "node_id"]).mean()
    mn_P_in = mn_P_in[pandas.MultiIndex.from_frame(P_in.index.to_frame()[["edge_count", "node_id"]])]
    mn_P_in.index = P_in.index

    P_out = (P_out / mn_P_out).dropna()
    P_in = (P_in / mn_P_in).dropna()

## Plotting the results

Here, we plot the results. 
From the pre-digested representation of the data we created above, a plot can be (relatively) rapidly created.

We make the plot customizable. 

**Read these descriptions carefully to understand the plots**
  - direction: Consider the effect on "incoming" or "outgoing" connectivity.
  - required_count: Minimum number of neuron pairs for a valid connection probability estimate in a spatial bin. It can be argued that a connection probability calculated from a single pair of neurons is meaningless. Increase this value to avoid that.
  - required_corr: Minimum number of data points for a valid estimate or Pearson correlation. It can be argued that correlations between 2 samples are meaningless. Increase to avoid this.
  - thresh_pair: By default, a pair of neurons is considered connected if there is at least 1 synapse between them. But you can increase this to require 2, 3, or more synapses to investigate the spatial structure of stronger connections
  - bin_xz and bin_y: Select one spatial bin using the two dropdowns. What will be displayed is the Pearson correlations of connection probabilities into the selected bin and connection probabilities into *all possible* bins. The location of the selected bin will be indicated by a blue "x" mark. The correlation with the selected bin will always be 1.0 by definition.
  - clim_max: Adjust how tight the limits of the color bar are set.


In [ ]:
def make_extent(df):
    delta_xz = df.columns[-1] - df.columns[-2]
    delta_y = df.index[-1] - df.index[-2]
    
    extent = [df.columns[0] - delta_xz/2, df.columns[-1] + delta_xz/2,
             df.index[-1] + delta_y/2, df.index[0] - delta_y/2]
    return extent

def image_plot_function(direction, required_count, required_corr, thresh_pair, bin_xz, bin_y, clim_max):
    if direction == "incoming":
        N = N_in; P = P_in
    elif direction == "outgoing":
        N = N_out; P = P_out
    
    idx = N.index[N >= required_count]
    src_bin = (bin_xz, bin_y)

    fig = plt.figure(figsize=(2.5, 3.5))
    ax = fig.gca()
    ptgt = P[thresh_pair].reindex(idx, fill_value=0)
    if src_bin not in ptgt:
        ax.text(0.5, 0.5, "Source bin empty!", horizontalalignment="center")
        return
    psrc = ptgt[src_bin]

    def cc(data_in):
        data_in = data_in.droplevel([0, 1])
        idx = psrc.index.intersection(data_in.index)
        if len(idx) > required_corr:
            return numpy.corrcoef(data_in[idx], psrc[idx])[0, 1]
        return numpy.nan
    I = ptgt.groupby(["xz", "y"]).apply(cc, include_groups=False)
    I = I.sort_index().unstack("xz")
    img = plt.imshow(I, clim=[-clim_max, clim_max],
                     cmap="coolwarm", extent=make_extent(I))
    plt.colorbar(img, label="Pearson corr.")
    # dy = y(post) - y(pre). If y is depth then a values > 0 indicates a _downwards_ connection.
    # Hence, for "Incoming" we want values > 0 towards the top of the plot. if y is not depth,
    # then the other way around.
    if y_is_depth == (direction == "outgoing"): # This is the opposite of part 2. I know. I apologize. But this is correct.
        ax.set_ylim(sorted(ax.get_ylim()))
    ax.plot(bin_xz, bin_y, marker='x', color="blue")
    ax.set_xlabel("hor. offset")
    ax.set_ylabel("vert. offset")

sel_direction = widgets.Dropdown(options=["incoming", "outgoing"], value="outgoing")
sel_required_count = widgets.IntSlider(min=1, max=250, value=50)
sel_required_corr = widgets.IntSlider(min=2, max=100, value=10)
sel_thresh_pair = widgets.IntSlider(min=1, max=3, value=1)
sel_xz = widgets.Dropdown(options=sorted(bin_centers_xz), index=1)
sel_y = widgets.Dropdown(options=sorted(bin_centers_y), index=int(len(bin_centers_y)/2))
sel_clim = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.5)

interactive(image_plot_function, direction=sel_direction, required_count=sel_required_count,
            required_corr=sel_required_corr,
            thresh_pair=sel_thresh_pair, bin_xz=sel_xz, bin_y=sel_y, clim_max=sel_clim)



## Cluster structure of correlations

Here, we perform one last analysis. We cluster the structure of correlations of spatial bins and depict the results. This reveals which spatial bins tend to be innervated together!

**Options**
  - direction: Consider the effect on "incoming" or "outgoing" connectivity.
  - required_count: Minimum number of neuron pairs for a valid connection probability estimate in a spatial bin. It can be argued that a connection probability calculated from a single pair of neurons is meaningless. Increase this value to avoid that.
  - thresh_pair: By default, a pair of neurons is considered connected if there is at least 1 synapse between them. But you can increase this to require 2, 3, or more synapses to investigate the spatial structure of stronger connections
  - louvain_res: Value of the resolution parameter of Louvain clustering.
  - min_cluster_sz: Cluster that comprise fewer than that number of spatial bins will be discarded

In [ ]:
from sknetwork.clustering import Louvain
from sknetwork.clustering import get_modularity
from scipy import stats

progress = widgets.IntProgress(min=0, max=10, value=0, description="Calculating...")
modularity_display = widgets.HTML(
    value="...",
    description='Modularity',
)

def cluster_and_show(direction, required_count, thresh_pair, louvain_res, min_cluster_sz):
    progress.value = 0
    if direction == "incoming":
        N = N_in; P = P_in
    elif direction == "outgoing":
        N = N_out; P = P_out

    fig = plt.figure(figsize=(2.5, 3.5))
    ax = fig.gca()

    idx = N.index[N >= required_count]
    ptgt = P[thresh_pair].reindex(idx, fill_value=0)
    progress.value = 2

    cc = ptgt.sort_index().unstack("node_id").transpose().corr().fillna(0)
    cc[cc < 0] = 0
    progress.value = 7

    res = Louvain(louvain_res).fit_predict(cc.values)
    clst = pandas.Series(res, index=cc.index)

    vc = clst.value_counts()
    invalid = vc.index[vc < min_cluster_sz]
    clst[clst.isin(invalid)] = numpy.nan
    I = clst.unstack("xz")
    progress.value = 9

    ax.imshow(I, extent=make_extent(I), cmap="Set3")
    ax.set_xlabel("hor. offset")
    ax.set_ylabel("vert. offset")
    if y_is_depth == (direction == "outgoing"): # This is the opposite of part 2. I know. I apologize. But this is correct.
        ax.set_ylim(sorted(ax.get_ylim()))

    modularity_display.value = f"<b>{get_modularity(cc.values, res, resolution=louvain_res)}</b>"
    progress.value = 10

sel_direction = widgets.Dropdown(options=["incoming", "outgoing"], value="outgoing")
sel_required_count = widgets.IntSlider(min=1, max=250, value=50, behavior="tap")
sel_thresh_pair = widgets.IntSlider(min=1, max=3, value=1, behavior="tap")
sel_louvain = widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, behavior="tap")
sel_min_sz = widgets.IntSlider(min=1, max=200, value=20, behavior="tap")

display(progress)
display(modularity_display)
interactive(cluster_and_show, direction=sel_direction, required_count=sel_required_count,
            thresh_pair=sel_thresh_pair, louvain_res=sel_louvain, min_cluster_sz=sel_min_sz)



## Interpreting the results
After running the analysis for all three connectomes and comparing them we can interpret the results.

We note:
  - There is a significant spatial extent to the correlations of connection probabilities for outgoing connections. In general, if a spatial bin is strongly innervated, the connection probabilities for bins ~150 um around it are elevated.
  - For incoming connectivity the effect is present, but much weaker.
  - In the data (without normalization), correlations are to the largest extent only positive. This contradicts a conceivable model where presence of axon at one location decreases its presence in other locations, because there is a limited amount of "energy" to spend on each axon. Instead it is compatible with a multiplicative model as proposed by Piazza and colleagues (https://doi.org/10.1101/2025.02.27.640551) 
  - The configuration model depicts strong correlations as well, but without specific spatial structure. This confirms the idea that there is a "global" effect that results from long-tailed degree distributions. However, correlations in the actual data has more specific spatial structure indicating that there is also a significant "local" effect. 
  - The "local" effect can be succesfully separated and visualized when the normalization is applied. The local effect also depicts negative correlations.
  - There appears to be a weak(!) local effect visible after normalization even for the configuration model. This is unexpected. It can be explained by the edge effect of connectivity: Neurons near the boundaries of the volumes will have a lower degree as many of their connections are extrinsic, leading to a weak interaction between degree and locations of connections.